In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [26.8 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-08 17:49:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-01-08 17:49:33 (11.0 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

# Deliverable 1 :An Amazon Review dataset is extracted as a DataFrame and extracted dataset is transformed into four DataFrames with the correct columns,All four DataFrames are loaded into their respective tables in pgAdmin 

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

### Create DataFrames to match tables

In [6]:
df=df.dropna()

In [7]:
# Removed duplicate rows
df= df.dropDuplicates()


In [8]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
# df_review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
# df_review_id_table.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1007YJJIFAC9J|   52769212|B009UHTGT0|     245155553| 2013-08-27|
|R100F8VKMH0GML|   22075855|B006Z96OI2|     556998710| 2015-02-28|
|R100MQXHRT2RWR|   49362036|B005PK7RW4|     475331940| 2015-06-07|
|R100Q7Y0SUHCK9|   22694784|B007UZNO4W|     240118513| 2014-08-10|
|R100QVXPVZS0DZ|   30117376|B00GZRPIEU|     206581276| 2015-01-14|
|R100UJWTF4UIOK|   43059193|B001TIOLN4|      98283682| 2013-02-12|
|R100ZA6RFJ5RUC|   10191404|B001VEB5AI|     374144182| 2012-12-05|
|R10129K7CS35V1|   24593931|B0057D81QK|     836654866| 2013-06-24|
|R101Q7XWPBH68Z|    4536548|B00CFSM670|     316045886| 2014-12-24|
|R101QDVAUUPBED|   50619433|B00020HQQE|     148453171| 2012-05-22|
|R101TUWPZXRJU3|   13146526|B000FOA7C2|       3633715| 2010-06-14|
| R101X9S0NAHJ1|   33439221|B00GXKVEJM|     833159451| 2015-08

In [29]:
from pyspark.sql.functions import col, to_date, to_timestamp
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

df_review_id_table = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [35]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
df_customers_table= df.groupby("customer_id").agg({"customer_id":"count"})

In [37]:
df_customers_table= df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")

In [38]:
df_customers_table.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45641495|             6|
|   48343488|             1|
|   17171509|            18|
|    7255220|             2|
|   10262401|             2|
+-----------+--------------+
only showing top 5 rows



In [40]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([])
df_products_table  = df.select(["product_id", "product_title"]).drop_duplicates()
df_products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00340NV90|New Massage Table...|
|B00N2BW2PK|Fitbit Charge HR ...|
|B00068JKNO|Swanson Coq10 100...|
|B001D8ZGAM|Aquasentials Easy...|
|B000PHFGV6|Tantus A Bomb Ana...|
|B00AWJBU2Q|13 mm Anti-slip S...|
|B004L9U71I|Purex Crystals In...|
|B000X2DOJE|Gelco GSEAT ULTRA...|
|B00TFYRBJ0|Res-Q 1250 Omega-...|
|B002E7N1CG|Nature Made High ...|
|B0011Y64RC|Himalaya Herbal H...|
|B0057RWSZQ|Quest Nutrition P...|
|B000GU4TXI|Allergy Research ...|
|B00847VUGE|TLS CORE Fat & Ca...|
|B003YQYTX6|CryoDerm Gel Size...|
|B00AYIM9Y8|BulkSupplements P...|
|B00BLNB1X0|Swanson Saffron E...|
|B000DZKQ5G|Battle Creek Ther...|
|B00Y34E9ZQ|Slight Touch Cont...|
|B000GCKA16|Boston Original C...|
+----------+--------------------+
only showing top 20 rows



In [41]:
df_review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1007YJJIFAC9J|   52769212|B009UHTGT0|     245155553| 2013-08-27|
|R100F8VKMH0GML|   22075855|B006Z96OI2|     556998710| 2015-02-28|
|R100MQXHRT2RWR|   49362036|B005PK7RW4|     475331940| 2015-06-07|
|R100Q7Y0SUHCK9|   22694784|B007UZNO4W|     240118513| 2014-08-10|
|R100QVXPVZS0DZ|   30117376|B00GZRPIEU|     206581276| 2015-01-14|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [42]:
df_review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [43]:
# Create the vine_table. DataFrame
df_vine_table = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
df_vine_table.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1007YJJIFAC9J|          5|            0|          0|   N|                Y|
|R100F8VKMH0GML|          5|            0|          0|   N|                Y|
|R100MQXHRT2RWR|          1|            0|          0|   N|                Y|
|R100Q7Y0SUHCK9|          4|            0|          0|   N|                N|
|R100QVXPVZS0DZ|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [44]:
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52769212|R1007YJJIFAC9J|B009UHTGT0|     245155553|  Scrub Daddy 8 Pack|Health & Personal...|          5|            0|          0|   N|                Y|Just As Good As t...|Reviewed this bef...| 2013-08-27|
|         US|   22075855|R100F8VKMH0GML|B006Z96OI2|     556998710|Marquee 5-Piece A...|Health & Personal

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [45]:
df_review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [46]:
df_products_table.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [47]:
df_customers_table.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [48]:
df_vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [49]:
# Configure settings for RDS
from getpass import getpass
password=getpass("Enter Aws DB pwd :")
mode = "append"

jdbc_url="jdbc:postgresql://uofa-bigdata-db.c9ch8w1dhnq3.us-west-1.rds.amazonaws.com:5432/amazon_vine"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter Aws DB pwd :··········


In [31]:
# Write review_id_df to table in RDS

df_review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [50]:
# Write products_df to table in RDS
# about 3 min
df_products_table.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [51]:
# Write customers_df to table in RDS
# 5 min 14 s
df_customers_table.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [52]:
# Write vine_df to table in RDS
# 11 minutes
df_vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)